In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys
path = '/content/drive/MyDrive/Colab Notebooks/NeurIPS2024'
sys.path.append(path)

import numpy as np
import pandas as pd
import pickle
import os

from realdata import *

import matplotlib.pyplot as plt


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
algo = 'XGB'

test_next = False


results_all = []
for idx in range(10):
    le = idx * 10 + 1
    ri = le + 9
    filename = path + '/results/{}_{}_{}_{}.pkl'.format(algo, le, ri, test_next)
    with open(filename, 'rb') as r:
        tmp = pickle.load(r)
        results_all += tmp


D = 10
M = int(np.ceil(826 / D))
list_t = np.array(range(1, M)) * D

prop = [0.3, 0.1]
window_sizes_train = [1, 16, 256, 1024]
window_sizes_infer = [1, 4, 16, 64, 256, 1024]
weights = [0.99, 0.9, 0.5, 0.25]
alpha = 0.1
delta = 0.1



num_train = len(window_sizes_train)
num_infer = len(window_sizes_infer)
num_weights = len(weights)
num_t = len(list_t)


S = len(results_all)
idx_start = 0
print('Algorithm = {}'.format(algo))
print()
print()

for (idx_model, window_size) in enumerate(window_sizes_train):
    print('Results for training window {}: '.format(window_sizes_train[idx_model]))

    MAE = dict()
    MAE['ARW'] = 0
    MAE['weighted'] = np.zeros(num_weights)
    MAE['fixed'] = np.zeros(num_infer)

    MAEsd = dict()
    MAEsd['ARW'] = 0
    MAEsd['weighted'] = np.zeros(num_weights)
    MAEsd['fixed'] = np.zeros(num_infer)

    AC = dict()
    AC['ARW'] = 0
    AC['weighted'] = np.zeros(num_weights)
    AC['fixed'] = np.zeros(num_infer)

    AW = dict()
    AW['ARW'] = 0
    AW['weighted'] = np.zeros(num_weights)
    AW['fixed'] = np.zeros(num_infer)
    AW['baseline'] = 0


    for s in range(S):
        MAE_s, AC_s, AW_s = summarize(results_all[s], alpha, idx_start)
        for key in MAE.keys():
            MAE[key] += MAE_s[idx_model][key] / S
            MAEsd[key] += MAE_s[idx_model][key] ** 2 / S
            AC[key] += AC_s[idx_model][key] / S
        for key in AW.keys():
            AW[key] += AW_s[idx_model][key] / S

    for key in MAE.keys():
        MAEsd[key] = np.sqrt(MAEsd[key] - (MAE[key] ** 2))

    print('MAE: ', MAE)
    print('AC: ', AC)
    print('AW: ', AW)

    print()

    print('MAEsd: ', MAEsd)

    print()
    print()


Algorithm = XGB


Results for training window 1: 
MAE:  {'ARW': 0.03426458643692904, 'weighted': array([0.04464658, 0.04467189, 0.04515827, 0.04581454]), 'fixed': array([0.06263844, 0.0366613 , 0.02996795, 0.03181962, 0.04122604,
       0.04464658])}
AC:  {'ARW': 0.9252327847018377, 'weighted': array([0.9395881 , 0.93962949, 0.94035529, 0.94122066]), 'fixed': array([0.87435701, 0.90336332, 0.91045881, 0.91873498, 0.93426842,
       0.9395881 ])}
AW:  {'ARW': 1.5807147116161602, 'weighted': array([1.69560077, 1.69588726, 1.70201203, 1.70993246]), 'fixed': array([1.37497605, 1.47014505, 1.49365196, 1.54790287, 1.65250323,
       1.69560077]), 'baseline': 2.175496702154525}

MAEsd:  {'ARW': 0.00219679387327899, 'weighted': array([0.00223704, 0.00223529, 0.00225445, 0.00224336]), 'fixed': array([0.00599437, 0.00343782, 0.00222995, 0.00223201, 0.00235977,
       0.00223704])}


Results for training window 16: 
MAE:  {'ARW': 0.03129912839813933, 'weighted': array([0.06184078, 0.06186269, 0.0